In [1]:
import pandas as pd
import numpy as np
import json
import math
import random
import datetime
import re
from collections import Counter
from utils import flatten
from nltk.tokenize import sent_tokenize

In [2]:
# JSON file from parsing the results of the qualtrics survey designed for extracting phenotypes from text.
QUALTRICS_RAW_CSV_EXPORT_PATH = "../qualtrics/results_raw_qualtrics_exports/SNPedia Survey_September 22, 2020_13.29.csv"
QUALTRICS_RESULTS_JSON_PATH = "../qualtrics/results_processed_json_files/snpedia_survey_september_22_2020_h13_m29.json"

# What the survey was built from, the IDs in this refer to unique text strings so that duplicates weren't part of it.
SURVEY_SOURCE_DATA_PATH = "../data/4_binned_and_blocked_texts.csv"

# This is the original data with duplicate text strings because IDs refer to particular SNPs from particular genes.
ORIGINAL_DATA_PATH = "../data/2_snps_and_cleaned_text.csv"

# Where to send the resulting output dataframes as CSV files.
random_six_digit_number = random.randrange(100000,999999)
datetime_str = datetime.datetime.now().strftime('%m_%d_%Y_h%Hm%Ms%S')

FILENAMES_OUTPUT_PATH = "../data/S{}_input_file_paths_{}.txt".format(random_six_digit_number, datetime_str)
RESPONSES_OUTPUT_PATH = "../data/S{}_processed_survey_responses_{}.csv".format(random_six_digit_number, datetime_str)
SURVEY_TAKERS_OUTPUT_PATH = "../data/S{}_processed_survey_takers_{}.csv".format(random_six_digit_number, datetime_str)
SNPS_AND_SNIPPETS_OUTPUT_PATH = "../data/S{}_snps_and_snippets_{}.csv".format(random_six_digit_number, datetime_str)
SNPS_AND_CONTEXTS_OUTPUT_PATH = "../data/S{}_snps_and_contexts_{}.csv".format(random_six_digit_number, datetime_str)

In [3]:
with open(FILENAMES_OUTPUT_PATH, "w") as f:
    filenames = [
        ORIGINAL_DATA_PATH,
        SURVEY_SOURCE_DATA_PATH,
        QUALTRICS_RAW_CSV_EXPORT_PATH,
        QUALTRICS_RESULTS_JSON_PATH]
    filenames = "\n".join(filenames)
    f.write(filenames)

In [4]:
# Make sure the mapping between the unique texts used to create the survey and the original SNP data exists.
original_df = pd.read_csv(ORIGINAL_DATA_PATH)
survey_df = pd.read_csv(SURVEY_SOURCE_DATA_PATH)
text_to_unique_text_id = dict(zip(survey_df.text, survey_df.id))
unique_text_id_to_text = dict(zip(survey_df.id, survey_df.text))
original_df["id"] = original_df["text"].map(text_to_unique_text_id)
original_df.head(20)

,gene,snp,text,id
0,AANAT,Rs28936679,"rs28936679, also known as Ala129Thr or A129T (...",1.0
1,AANAT,Rs3760138,Genetic differences in human circadian clock g...,2.0
2,AANAT,Rs4238989,Genetic differences in human circadian clock g...,3.0
3,ABCA1,Rs1800977,The -14C->T polymorphism rs1800977 of the ABCA...,4.0
4,ABCA1,Rs1883025,Apolipoprotein E levels in cerebrospinal fluid...,5.0
5,ABCA1,Rs2020927,"rs2297404, rs2230808, and rs2020927 haplotype ...",6.0
6,ABCA1,Rs2066714,Apolipoprotein E levels in cerebrospinal fluid...,7.0
7,ABCA1,Rs2066715,Apolipoprotein E levels in cerebrospinal fluid...,8.0
8,ABCA1,Rs2230806,"rs2230806, also known as Arg219Lys or R219K, i...",9.0
9,ABCA1,Rs2230808,"rs2297404, rs2230808, and rs2020927 haplotype ...",10.0


In [5]:
# Create a dictionary using the JSON data output from a qualtrics survey.
with open(QUALTRICS_RESULTS_JSON_PATH) as f:
    responses = json.load(f)
responses

[{'response_id': 'R_eOSosrnczqYqE9P',
  'recorded_datetime': '2020-09-22 10:37:23',
  'status': 'IP Address',
  'progress': 100,
  'is_finished': 'True',
  'duration': 1252,
  'hilights': [{'qid': 4769, 'selection': 'Disease ', 'selection_index': 13},
   {'qid': 5162,
    'selection': 'Focal segmental glomerulosclerosis',
    'selection_index': 0},
   {'qid': 2451, 'selection': ' Alpha Thalassemia', 'selection_index': 23},
   {'qid': 3964,
    'selection': 'Congenital Disorder of Glycosylation',
    'selection_index': 0},
   {'qid': 5448, 'selection': 'schizophrenia.', 'selection_index': 54},
   {'qid': 3027, 'selection': 'glaucoma', 'selection_index': 29},
   {'qid': 2287,
    'selection': 'Nonsyndromic Sensorineural',
    'selection_index': 33},
   {'qid': 2287, 'selection': 'Connexin', 'selection_index': 13},
   {'qid': 1783, 'selection': 'schizophrenia', 'selection_index': 10},
   {'qid': 628, 'selection': 'cholesterol', 'selection_index': 4},
   {'qid': 628, 'selection': 'choleste

In [6]:
# Load the original source CSV that was used in creating the survey. This will used to check against the results.
# We want to make sure there is no discrepency in which IDs are referring to which texts.
source_df = pd.read_csv(SURVEY_SOURCE_DATA_PATH)
#source_df.reset_index(drop=False, inplace=True)

# Question IDs in the actual survey will use the 
#source_df["qid"] = source_df["index"]
#source_df.drop(labels=["index"], axis=1, inplace=True)
#qid_to_source_text = {i:text for i,text in zip(source_df["qid"].values, source_df["text"].values)}
#qid_to_unique_text_id = {qid:i for qid,i in zip(source_df["qid"].values, source_df["id"].values)}
source_df.head(10)

,id,text,bin_id,bin_size,block_id,block_size,block_sample
0,1169,smoking,1,988,1,247,5
1,3702,Phenylketonuriars62514952,1,988,1,247,5
2,3701,Phenylketonuriars5030860,1,988,1,247,5
3,3700,Phenylketonuriars5030859,1,988,1,247,5
4,3699,Phenylketonuriars5030856,1,988,1,247,5
5,3698,Phenylketonuriars5030851,1,988,1,247,5
6,3697,Phenylketonuriars5030850,1,988,1,247,5
7,3696,Phenylketonuriars5030847,1,988,1,247,5
8,3695,Phenylketonuriars5030846,1,988,1,247,5
9,3694,Phenylketonuriars5030843,1,988,1,247,5


In [7]:
# Put the responses into a dataframe.
row_tuples = []
for response in responses:
    
    # Metadata associated with this response.
    response_id = response["response_id"]
    recorded_datetime = response["recorded_datetime"]
    status = response["status"]
    progress = response["progress"]
    is_finished = response["is_finished"]
    duration = response["duration"]


    # The actual highlighted text strings from this response.
    for hilight in response["hilights"]:

        # The information about this one particular highlight.
        qid = hilight["qid"]
        hilighted_text = hilight["selection"]
        index_of_first_selected_char = hilight["selection_index"]
        source_text = unique_text_id_to_text[qid]        

        # First check that the question IDs are correct so we know what the source text was for this question.
        # Then additionally make sure the location of the highlight makes sense as well.
        #assert hilighted_text in source_text
        #assert source_text[index_of_first_selected_char:index_of_first_selected_char+len(hilighted_text)] == hilighted_text
       
        # Those asserts don't pass 100% of the time due to special cases with the text.
        # Save them to variables instead and check how often they don't pass, should be very infrequently.
        # Look at the special cases by hand.
        text_match = (hilighted_text in source_text)
        indices_match = (source_text[index_of_first_selected_char:index_of_first_selected_char+len(hilighted_text)] == hilighted_text)
        
        # Add this as a row.
        row_tuples.append((response_id, recorded_datetime, status, progress, is_finished, duration, qid, hilighted_text, text_match, indices_match))

columns = ["response_id", "recorded_datetime", "status", "progress", "is_finished", "duration", "id", "snippet", "text_match", "idx_match"]
df = pd.DataFrame(row_tuples, columns=columns)
df.to_csv(RESPONSES_OUTPUT_PATH, index=False)
df.head(10)

,response_id,recorded_datetime,status,progress,is_finished,duration,id,snippet,text_match,idx_match
0,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,4769,Disease,True,True
1,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,5162,Focal segmental glomerulosclerosis,True,True
2,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,2451,Alpha Thalassemia,True,True
3,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,3964,Congenital Disorder of Glycosylation,True,True
4,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,5448,schizophrenia.,True,True
5,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,3027,glaucoma,True,True
6,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,2287,Nonsyndromic Sensorineural,True,True
7,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,2287,Connexin,True,True
8,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,1783,schizophrenia,True,True
9,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,628,cholesterol,True,True


In [8]:
# How many times did the asserts not evaluate to true?
print(Counter(df["text_match"].values))
print(Counter(df["idx_match"].values))

Counter({True: 1564, False: 8})
Counter({True: 1468, False: 104})


In [9]:
# Preparing a dataset of context sentences that include the highlighted text snippets.
def get_contexts(i, snippet, text):
    sentences = sent_tokenize(text)
    sentences_with_snippet = [s for s in sentences if snippet in s]
    
    # Some additional processing to clean up the context sentences. Can insert more steps here if needed.
    snp_pattern = re.compile(r"rs[0-9]+")
    whitespace_pattern = re.compile(r"\s+")
    whitespace_before_period_pattern = re.compile(r"\s+[\.\;]")
    whitespace_before_comma_pattern = re.compile(r"\s+[\.\;]")
    sentences_with_snippet = [snp_pattern.sub(r" [SNP] ", s) for s in sentences_with_snippet]
    sentences_with_snippet = [s.replace("("," (") for s in sentences_with_snippet]
    sentences_with_snippet = [s.replace(")",") ") for s in sentences_with_snippet]
    sentences_with_snippet = [whitespace_pattern.sub(r" ", s) for s in sentences_with_snippet]
    sentences_with_snippet = [whitespace_before_period_pattern.sub(r".", s) for s in sentences_with_snippet]
    sentences_with_snippet = [whitespace_before_comma_pattern.sub(r".", s) for s in sentences_with_snippet]
    sentences_with_snippet = [s.strip() for s in sentences_with_snippet]
    sentences_with_snippet = ["{}{}".format(s[0].upper(),s[1:]) for s in sentences_with_snippet]
    add_end_character = lambda x: "{}.".format(x[:len(x)-1]) if (x[len(x)-1]==".") or (x[len(x)-1]==";") else "{}.".format(x)
    sentences_with_snippet = [add_end_character(s) for s in sentences_with_snippet]
    sentences_with_snippet = [s.replace('"', '') for s in sentences_with_snippet]
    sentences_with_snippet = [s.replace("'", "") for s in sentences_with_snippet]
    return((i,sentences_with_snippet))

# Create a mapping between text IDs and lists of the context sentences that relevant snippets were found in.
obj = df.apply(lambda row: get_contexts(row.id, row.snippet, unique_text_id_to_text[row.id]), axis=1)
id_to_context_sentences = dict(obj.values)
id_to_context_sentences

{4769: ['Niemann-Pick Disease Type A [SNP].'],
 5162: ['Focal segmental glomerulosclerosis 2 [SNP].'],
 2451: ['[SNP] see HBA2 and Alpha Thalassemia.'],
 3964: ['Congenital Disorder of Glycosylation Type 1a [SNP].'],
 5448: ['Evidence of sex-modulated association of ZNF804A with schizophrenia.'],
 3027: ['Association with exfoliation glaucoma; possible genoset partner with [SNP].'],
 2287: ['Hearing loss?Connexin 26-Related Nonsyndromic Sensorineural Hearing Losssee [SNP].'],
 1783: ['Linked to schizophrenia with [SNP] , [SNP] , and [SNP].'],
 628: ['LDL cholesterol and total cholesterol levels being the quantitative trait associated with in.'],
 2515: ['Hemochromatosis related; better known in dbSNP nomenclature as [SNP] / discussion at 23andMe.'],
 3684: ['[SNP] increases susceptibility to Rheumatoid Arthritis 1.89 times for carriers of the G allele.'],
 4613: ['[SNP] and [SNP] replicated for rheumatoid arthritis in Japanese, but not Caucasian populations.'],
 2261: ['GTP cyclohydrol

In [10]:
# We need the unprocessed CSV file exported from Qualtrics for some of the following information.
# Specifically, let's use this to get the mapping betweeen response IDs (from Qualtrics) and user IDs (from Prolific).
exported_df = pd.read_csv(QUALTRICS_RAW_CSV_EXPORT_PATH)
exported_df.drop(df.index[[0,1]], inplace=True)
prolific_pid_column = "Q564"
response_id_column = "ResponseId"
response_id_to_prolific_pid = dict(zip(exported_df[response_id_column], exported_df[prolific_pid_column]))
response_id_to_prolific_pid

{'R_eOSosrnczqYqE9P': '5f56f670e492b316bbd2a47c',
 'R_2XjJU6SMJAlU4I2': '5e703f3dd6d4336135b8818d',
 'R_25R9SJyLb4p6iv7': '5f68eb472f60ad099401f587',
 'R_1CpRIVeWQ8AGNqi': '5f206e18fb9d281e91500fcd',
 'R_2wQNxJNyQdjSeq9': '5d3cc9010e510a00013df6f6',
 'R_qPe5vObZ349P3I5': '5ec5925cb3d6035afea34d20',
 'R_2Bs2kBIw93NmaCj': '5e7e6df49867d653e250ade8',
 'R_2wodESRdFsJZ5pq': '5f3e31e2769ffb267b9d1a24',
 'R_PUpvfu7l6QKh6mJ': '5f4da07b144b1ca7f76c51ed'}

In [11]:
# Breaking it down by specific users who took the survey.
users_df = df.copy(deep=True)[["response_id","recorded_datetime","status","progress","is_finished","duration"]]
users_df.drop_duplicates(inplace=True)
users_df.reset_index(inplace=True, drop=True)

# Some extra information to know, the duration, and the number of total highlights in each response, and Prolific IDs.
users_df["duration_min"] = users_df["duration"].map(lambda x: math.ceil(x/60))
response_id_to_num_snippets = dict(df.groupby("response_id").size())
users_df["num_snippets"] = users_df["response_id"].map(response_id_to_num_snippets)
users_df["prolific_id"] = users_df["response_id"].map(response_id_to_prolific_pid)
users_df.to_csv(SURVEY_TAKERS_OUTPUT_PATH, index=False)
users_df

,response_id,recorded_datetime,status,progress,is_finished,duration,duration_min,num_snippets,prolific_id
0,R_eOSosrnczqYqE9P,2020-09-22 10:37:23,IP Address,100,True,1252,21,96,5f56f670e492b316bbd2a47c
1,R_2XjJU6SMJAlU4I2,2020-09-22 10:40:27,IP Address,100,True,1074,18,120,5e703f3dd6d4336135b8818d
2,R_1CpRIVeWQ8AGNqi,2020-09-22 10:45:55,IP Address,100,True,2002,34,171,5f206e18fb9d281e91500fcd
3,R_2wQNxJNyQdjSeq9,2020-09-22 11:20:01,IP Address,100,True,1438,24,145,5d3cc9010e510a00013df6f6
4,R_qPe5vObZ349P3I5,2020-09-22 11:32:04,IP Address,100,True,3442,58,334,5ec5925cb3d6035afea34d20
5,R_2Bs2kBIw93NmaCj,2020-09-22 12:03:12,IP Address,100,True,2096,35,235,5e7e6df49867d653e250ade8
6,R_2wodESRdFsJZ5pq,2020-09-22 12:13:30,IP Address,100,True,2665,45,248,5f3e31e2769ffb267b9d1a24
7,R_PUpvfu7l6QKh6mJ,2020-09-22 12:35:56,IP Address,100,True,4449,75,223,5f4da07b144b1ca7f76c51ed


In [12]:
# Create a mapping from unique text IDs to all of the text snippets that were hilighted in these survey results.
unique_text_id_to_snippet_list = {}
for unique_text_id,row_indices in df.groupby("id", axis=0).groups.items():
    hilighted_texts_list = list(df.iloc[row_indices]["snippet"].values)
    unique_text_id_to_snippet_list[unique_text_id] = hilighted_texts_list
print(unique_text_id_to_snippet_list)

{4: ['polymorphism', 'therothrombotic cerebral infarction'], 15: ['congenital Lamellar ichthyosis', 'skin condition'], 21: ["Alzheimer's disease", "late-onset Alzheimer's disease "], 22: ["Alzheimer's disease"], 27: ['depression ', 'depression', 'depressio', 'colonic disease '], 28: ['Polymorphisms in genes that regulate cyclosporine metabolism affect cyclosporine blood levels and clinical outcomes in patients who receive allogeneic hematopoietic stem cell transplantation.', 'esophageal cancer.', ' Pharmacogenetics of antidepressant response. Correlation between genetic polymorphisms', 'rs1045642, also known as C3435T, is a SNP located in the ABCB1 gene. It is often studied in conjunction with rs2032582. C3435T has been mentioned by: A "Silent" Polymorphism in the MDR1 Gene Changes Substrate Specificity (for example, to verapamil) (R)-lansoprazole (Prevacid) concentrations are significantly increased in CYP2C19 extensive metabolizers with ABCB1 C3435T C allele. In a Korean population, 

In [13]:
# Use that mapping to create a version of the original dataframe with just the hilighted text snippets.
subset_df = original_df.copy(deep=True)[original_df["id"].isin(unique_text_id_to_snippet_list.keys())]
subset_df["n"] = subset_df["id"].map(lambda x: len(unique_text_id_to_snippet_list[x]))
text_snippets = flatten([unique_text_id_to_snippet_list[i] for i in subset_df["id"].values])

# Extend the dataframe to duplicate each row n times where n is the number of text hilight results from the surveys.
modified_df = subset_df.reindex(np.repeat(subset_df.index.values, subset_df["n"]), method="ffill")

# Make sure that the extension occured as expected based on the number of text snippets, and add them as a new column.
assert len(modified_df) == len(text_snippets)
modified_df["snippet"] = text_snippets
modified_df.head(20)

,gene,snp,text,id,n,snippet
3,ABCA1,Rs1800977,The -14C->T polymorphism rs1800977 of the ABCA...,4.0,2,polymorphism
3,ABCA1,Rs1800977,The -14C->T polymorphism rs1800977 of the ABCA...,4.0,2,therothrombotic cerebral infarction
14,ABCA12,Rs28940268,This is a recessive SNP for congenital Lamella...,15.0,2,congenital Lamellar ichthyosis
14,ABCA12,Rs28940268,This is a recessive SNP for congenital Lamella...,15.0,2,skin condition
15,ABCA12,Rs28940269,This is a recessive SNP for congenital Lamella...,15.0,2,congenital Lamellar ichthyosis
15,ABCA12,Rs28940269,This is a recessive SNP for congenital Lamella...,15.0,2,skin condition
16,ABCA12,Rs28940270,This is a recessive SNP for congenital Lamella...,15.0,2,congenital Lamellar ichthyosis
16,ABCA12,Rs28940270,This is a recessive SNP for congenital Lamella...,15.0,2,skin condition
17,ABCA12,Rs28940271,This is a recessive SNP for congenital Lamella...,15.0,2,congenital Lamellar ichthyosis
17,ABCA12,Rs28940271,This is a recessive SNP for congenital Lamella...,15.0,2,skin condition


In [14]:
# Save that dataframe as a new CSV file with just the final cleaned text snippets for each gene and SNP.
modified_df = modified_df[["gene","snp","snippet"]]
modified_df.sort_values(by="gene", inplace=True)
modified_df.to_csv(SNPS_AND_SNIPPETS_OUTPUT_PATH, index=False)
modified_df.to_csv("../genes_snps_snippets.tsv", sep="\t", index=False)
modified_df.head(20)

,gene,snp,snippet
3,ABCA1,Rs1800977,polymorphism
3,ABCA1,Rs1800977,therothrombotic cerebral infarction
18,ABCA12,Rs28940568,congenital Lamellar ichthyosis
17,ABCA12,Rs28940271,skin condition
17,ABCA12,Rs28940271,congenital Lamellar ichthyosis
16,ABCA12,Rs28940270,skin condition
18,ABCA12,Rs28940568,skin condition
15,ABCA12,Rs28940269,skin condition
15,ABCA12,Rs28940269,congenital Lamellar ichthyosis
14,ABCA12,Rs28940268,skin condition


In [15]:
# How many genes had atleast one snippet associated to it in these results?
len(modified_df["gene"].unique())

363

In [16]:
# Create a version of the dataset with mapping genes and SNPs to context sentences.
subset_df = original_df.copy(deep=True)[original_df["id"].isin(id_to_context_sentences.keys())]
subset_df["n"] = subset_df["id"].map(lambda x: len(id_to_context_sentences[x]))
sentences = flatten([id_to_context_sentences[i] for i in subset_df["id"].values])

# Extend the dataframe to duplicate each row n times where n is the number of text hilight results from the surveys.
modified_df = subset_df.reindex(np.repeat(subset_df.index.values, subset_df["n"]), method="ffill")

# Make sure that the extension occured as expected based on the number of text snippets, and add them as a new column.
assert len(modified_df) == len(sentences)
modified_df["context"] = sentences
modified_df.head(20)

,gene,snp,text,id,n,context
3,ABCA1,Rs1800977,The -14C->T polymorphism rs1800977 of the ABCA...,4.0,1,The -14C->T polymorphism [SNP] of the ABCA1 ge...
14,ABCA12,Rs28940268,This is a recessive SNP for congenital Lamella...,15.0,1,This is a recessive SNP for congenital Lamella...
15,ABCA12,Rs28940269,This is a recessive SNP for congenital Lamella...,15.0,1,This is a recessive SNP for congenital Lamella...
16,ABCA12,Rs28940270,This is a recessive SNP for congenital Lamella...,15.0,1,This is a recessive SNP for congenital Lamella...
17,ABCA12,Rs28940271,This is a recessive SNP for congenital Lamella...,15.0,1,This is a recessive SNP for congenital Lamella...
18,ABCA12,Rs28940568,This is a recessive SNP for congenital Lamella...,15.0,1,This is a recessive SNP for congenital Lamella...
24,ABCA7,Rs200538373,"rs200538373, also known as c.5570+5G>C, is one...",21.0,1,"[SNP] , also known as c.5570+5G>C, is one of s..."
25,ABCA7,Rs3764650,"Alzheimer's disease associated, based on large...",22.0,1,"Alzheimers disease associated, based on large ..."
47,ABCB4,Rs58238559,aka c.523A>G (p.Thr175Ala)Based on a 2015 stud...,44.0,1,Aka c.523A>G (p.Thr175Ala) Based on a 2015 stu...
52,ABCC2,Rs717620,Association of the multidrug-resistance-associ...,49.0,1,Polymorphisms of MRP2 (ABCC2) are associated w...


In [17]:
# Save that dataframe as a new CSV file with just the contexts for each gene and SNP.
modified_df = modified_df[["gene","snp","context"]]
modified_df.sort_values(by="gene", inplace=True)
modified_df.to_csv(SNPS_AND_CONTEXTS_OUTPUT_PATH, index=False)
modified_df.to_csv("../genes_snps_contexts.tsv", sep="\t", index=False)
modified_df.head(20)

,gene,snp,context
3,ABCA1,Rs1800977,The -14C->T polymorphism [SNP] of the ABCA1 ge...
14,ABCA12,Rs28940268,This is a recessive SNP for congenital Lamella...
15,ABCA12,Rs28940269,This is a recessive SNP for congenital Lamella...
16,ABCA12,Rs28940270,This is a recessive SNP for congenital Lamella...
17,ABCA12,Rs28940271,This is a recessive SNP for congenital Lamella...
18,ABCA12,Rs28940568,This is a recessive SNP for congenital Lamella...
24,ABCA7,Rs200538373,"[SNP] , also known as c.5570+5G>C, is one of s..."
25,ABCA7,Rs3764650,"Alzheimers disease associated, based on large ..."
47,ABCB4,Rs58238559,Aka c.523A>G (p.Thr175Ala) Based on a 2015 stu...
52,ABCC2,Rs717620,Polymorphisms of MRP2 (ABCC2) are associated w...
